In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
import random


model_name = 'eryk-mazus/polka-1.1b'
device = 'cuda'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [4]:
def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

def sentence_prob(sentence_txt):
    input_ids = tokenizer(sentence_txt, return_tensors='pt')['input_ids'].to(device)
    with torch.no_grad():
        output = model(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().numpy()    

In [35]:
corpus = [w.split("|") for w in """
wprost|wyprosty|wyprostu|wyprost uwielbiała|wielbił|wielbiła|uwielbił|wielbiło|uwielbiał|uwielbiało|uwielbiały
słuchać|osłuchać|słychać|usłuchać o|i|e|a|ó|ę|y|ą|u
wartościach własnych|owłosionych macierzy|mocarz|macierzą|macierze|mocarza|mocarze|mocarzy|macierz
""".split()]

In [36]:
from itertools import chain
def beam_search(k, corpus):
	
	best_k = corpus[0]
	for words in corpus[1:]:
		best_k = sorted(chain(*[
			sorted((f"{prefix} {w}" for w in words), key=sentence_prob, reverse=True)[:k]
			for prefix in best_k
		]), key=sentence_prob, reverse=True)[:k]
		print(best_k)


In [38]:
beam_search(k=10, corpus=corpus)

['wprost uwielbiał', 'wprost uwielbiała', 'wprost wielbił', 'wprost wielbiła', 'wprost uwielbiało', 'wprost uwielbiały', 'wyprost wielbił', 'wprost wielbiło', 'wyprost wielbiła', 'wyprost uwielbiał']
['wprost uwielbiał słuchać', 'wprost uwielbiała słuchać', 'wprost uwielbiało słuchać', 'wprost uwielbiały słuchać', 'wprost uwielbiał słychać', 'wprost uwielbiała słychać', 'wprost wielbił słuchać', 'wyprost uwielbiał słuchać', 'wprost uwielbiało słychać', 'wyprost wielbił słuchać']
['wprost uwielbiała słuchać o', 'wprost uwielbiał słuchać o', 'wprost uwielbiał słuchać i', 'wprost uwielbiała słuchać i', 'wprost uwielbiało słuchać o', 'wprost uwielbiały słuchać o', 'wprost uwielbiało słuchać i', 'wprost uwielbiał słuchać u', 'wprost uwielbiał słuchać a', 'wprost uwielbiała słuchać u']
['wprost uwielbiała słuchać o wartościach', 'wprost uwielbiał słuchać o wartościach', 'wprost uwielbiało słuchać o wartościach', 'wprost uwielbiały słuchać o wartościach', 'wprost uwielbiał słuchać i wartościa